In [13]:

import ee

# Trigger the authentication flow.
ee.Authenticate(auth_mode='localhost')



True

In [14]:

ee.Initialize(project="ee-aletschpy")

In [30]:
import datetime
import geemap.core as geemap
import json
import geojson
import geopandas as gpd
from AOI_polygon import load_polygon_from_geojson

In [67]:
ini_date=ee.Date('2025-01-01')
fin_date=ee.Date('2025-01-31')
polygon_path=r'C:\Users\marga\Documents\EPFL\Cours\S2\Design_project\codes\Satellite_glacier\draft\polygon\Aletsch.geojson'
with open(polygon_path, 'r') as file:
            geojson_data = geojson.load(file)
coords = geojson_data['features'][0]['geometry']['coordinates']

print(geojson_data)

aoi=ee.Geometry.Polygon(coords)

Map = geemap.Map()
# Add the image to the map.
Map.add_layer(aoi,None, 'Random area') #Map.addLayer(ee_object, vis_params, name, shown, opacity)
Map.centerObject(aoi, 11)


{"features": [{"geometry": {"coordinates": [[[8.029264, 46.38989], [8.083787, 46.415857], [8.112244, 46.455637], [8.10061, 46.493467], [8.084674, 46.502152], [8.075226, 46.528893], [8.072188, 46.545126], [8.057696, 46.555843], [8.03891, 46.55889], [8.01846, 46.563817], [7.999924, 46.55946], [7.986271, 46.553725], [7.963228, 46.544273], [7.960629, 46.528985], [7.964325, 46.522157], [7.965008, 46.501874], [7.939698, 46.490096], [7.940275, 46.467611], [7.981982, 46.468465], [7.997159, 46.450908], [7.997803, 46.439139], [7.984485, 46.405452], [8.030632, 46.386235], [8.029264, 46.38989]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}], "type": "FeatureCollection"}


<IPython.core.display.Javascript object>

In [104]:
Image_batch=ee.ImageCollection('COPERNICUS/S2_HARMONIZED') .filterBounds(aoi) .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
batch_head=Image_batch.limit(3)# 3 first images in the collection 
img = ee.Image( Image_batch
                       .first() 
                       .clip(aoi)
                       .select('B2','B3','B4','B8','B11'),)
Map.addLayer(img, None, 'First Copernicus image')
display(Map)

Map(bottom=1485144.0, center=[46.43523455938478, 7.8662900032168155], controls=(ZoomControl(options=['position…

# Cloud display

In [94]:
AOI = ee.Geometry.Point(-122.269, 45.701)
START_DATE = '2020-06-01'
END_DATE = '2020-06-02'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [95]:
def get_s2_sr_cld_col(aoi, start_date, end_date,CLOUD_FILTER):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [96]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE,CLOUD_FILTER)

In [97]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [98]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [99]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw) #return img.addBands(is_cld_shdw)

In [100]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [101]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [102]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

display_cloud_layers(s2_sr_cld_col_eval_disp)

# Download

In [105]:
"""Demonstrates the ee.Image.getDownloadURL method."""

import io
import requests
import ee


# A Sentinel-2 surface reflectance image.
#img = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')

# A small region within the image.
region = aoi #ee.Geometry.BBox(-122.0859, 37.0436, -122.0626, 37.0586)

# Image chunk as a NumPy structured array.
import numpy
url = img.getDownloadUrl({
    'bands': ['B3', 'B8', 'B11'],
    'region': region,
    'scale': 20,
    'format': 'NPY'
})
response = requests.get(url)
data = numpy.load(io.BytesIO(response.content))
print(data)
print(data.dtype)

# Single-band GeoTIFF files wrapped in a zip file.
# url = img.getDownloadUrl({
#     'name': 'single_band',
#     'bands': ['B3', 'B8', 'B11'],
#     'region': region
# })
# response = requests.get(url)
# with open('single_band.zip', 'wb') as fd:
#   fd.write(response.content)

# Multi-band GeoTIFF file wrapped in a zip file.
# url = img.getDownloadUrl({
#     'name': 'multi_band',
#     'bands': ['B3', 'B8', 'B11'],
#     'region': region,
#     'scale': 20,
#     'filePerBand': False
# })
# response = requests.get(url)
# with open('multi_band.zip', 'wb') as fd:
#   fd.write(response.content)

# Band-specific transformations.
# url = img.getDownloadUrl({
#     'name': 'custom_single_band',
#     'bands': [
#         {'id': 'B3', 'scale': 10},
#         {'id': 'B8', 'scale': 10},
#         {'id': 'B11', 'scale': 20}
#     ],
#     'region': region
# })
# response = requests.get(url)
# with open('custom_single_band.zip', 'wb') as fd:
#   fd.write(response.content)

# Multi-band GeoTIFF file.
url = img.getDownloadUrl({
    'bands': ['B3', 'B8', 'B11'],
    'region': region,
    'scale': 20,
    'format': 'GEO_TIFF'
})
response = requests.get(url)
with open('multi_band.tif', 'wb') as fd:
  fd.write(response.content)

[[(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]
 [(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]
 [(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]
 ...
 [(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]
 [(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]
 [(0, 0, 0) (0, 0, 0) (0, 0, 0) ... (0, 0, 0) (0, 0, 0) (0, 0, 0)]]
[('B3', '<u2'), ('B8', '<u2'), ('B11', '<u2')]
